# Hashing und Symmetrische Verschlüsselung

In diesem Notebook werden **zwei Hashverfahren** (SHA-2, SHA-3) und **zwei symmetrische Verschlüsselungsverfahren** (AES, Blowfish) auf einen Datensatz angewendet.  

Datensatz: *Employee Performance and Salary Dataset* (Beispiel mit Spalten ID, Name, Salary u.a.).


## Begründung der Auswahl der Verfahren

**Hashverfahren:**
- Der Hauptunterschied zwischen SHA2 und SHA3 ist das interne Algorithmusdesign, wenig Gemeinsamkeiten (außer den Namen). Daher deckt dieses Notebook beide Verfahren ab. Beide Verfahren gelten als sicher.
- **SHA-2 (SHA-256)**
  Der Hauptunterschied zwischen SHA2 und SHA3 ist das interne Algorithmusdesign, sie haben wenig bzw. keine Gemeinsamkeiten (außer den Namen). Daher deckt dieses Notebook beide Verfahren ab. Beide Verfahren gelten als sicher. SHA-2 ist in vielen sicherheitsrelevanten Anwendungen im Einsatz. SHA-2 ist im Vergleich zu SHA3 in vielen Implementierungen schneller, besonders auf älterer Hardware (https://piproductora.com/de/was-ist-der-unterschied-zwischen-sha-3-und-sha-256/)

- **SHA-3 (z. B. SHA3-256)** – neuer NIST-Standard, basiert auf einer anderen Konstruktion (wie Keccak). SHA-3 ist resistent gegen Längenerweiterungsangriffe
  SHA-3 wurde 2015 standardisiert und unterscheidet sich strukturell komplett von SHA-2. Laut NIST ist es nicht als Ersatz, sondern als zusätzliche Absicherung gedacht (https://stackoverflow.com/questions/14356526/whats-the-difference-between-the-hash-algorithms-sha-2-and-sha-3) Es ist resistent gegen Angriffstechniken, die theoretisch bei SHA-2 wirken könnten, und bietet mit SHAKE flexible Ausgabelängen (https://piproductora.com/de/was-ist-der-unterschied-zwischen-sha-3-und-sha-256/)

**Symmetrische Verschlüsselung:**
- **Blowfish** – schneller Algorithmus mit variabler Schlüssellänge, bei open source Verschlüsselungssoftware sehr beliebt. Allein für Implementierungen bei denen häufig neue Schlüssel erzeugt werden müssen, beispielsweise VPN, eignet sich Blowfish weniger. (https://www.infopoint-security.de/des-aes-blowfish-was-sind-die-sichersten-algorithmen-fuer-verschluesselungssoftware/a8366/)

- **AES (z. B. AES-256 im CBC-Modus)** – Industriestandard, sehr sicher, effizient und für große Datenmengen geeignet.  
  AES ist seit 2000 der offizielle Nachfolger von DES, weltweit der Standard in sicherheitskritischen Anwendungen (z. B. VPNs). Es arbeitet mit 128-Bit-Blöcken und 128/192/256-Bit Schlüsseln. AES ist sowohl sicher als auch performant und wird kontinuierlich geprüft (https://www.infopoint-security.de/des-aes-blowfish-was-sind-die-sichersten-algorithmen-fuer-verschluesselungssoftware/a8366/)

# Code

### Link zum Datensatz
https://www.kaggle.com/datasets/nadeemajeedch/employee-performance-and-salary-dataset/data?select=Employe_Performance_dataset.csv

Der Employee Performance and Salary Dataset enthält Informationen zu Mitarbeitenden wie Alter, Geschlecht, Abteilung, Gehalt, Eintrittsdatum, Leistung, Erfahrung, Status und Arbeitszeit.
Er umfasst sowohl numerische als auch kategoriale Daten und weist teilweise fehlende Werte bei der Leistungsbewertung auf.
Die Daten ermöglichen Analysen zu Trends in Gehalt, Leistung und Beschäftigungsstatus über verschiedene Zeiträume und Standorte hinweg.
Mit insgesamt 12 Spalten und zahlreichen Einträgen eignet sich der Datensatz gut für Aufgaben in den Bereichen Datenanalyse, Visualisierung und maschinelles Lernen.

In [2]:
! pip install -q kaggle

In [3]:
! mkdir ~/.kaggle

Übergabe von Kaggle Benutzer Daten:
{"username":"....","key":"...."}

In [4]:
# ... Hier ihre JSON Cred als dictionary eingeben
d_json_cred ={"username":"lizzldizzl","key":"7126d6d48a18986c8a8704fbb94e4a44"}

Kaggle Zugangsdaten speichern



In [5]:
import pandas as pd
kaggle_cred = pd.DataFrame(d_json_cred, index=[0]).to_json("~/.kaggle/kaggle.json")

Authorisierung geben dass Kaggle Daten heruntergeladen werden dürfen

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
!kaggle datasets download -d nadeemajeedch/employee-performance-and-salary-dataset

Dataset URL: https://www.kaggle.com/datasets/nadeemajeedch/employee-performance-and-salary-dataset
License(s): Community Data License Agreement - Sharing - Version 1.0
  0% 0.00/215k [00:00<?, ?B/s]
100% 215k/215k [00:00<00:00, 458MB/s]


Unzip der Daten

In [8]:
!unzip employee-performance-and-salary-dataset.zip -d ./data

Archive:  employee-performance-and-salary-dataset.zip
  inflating: ./data/50_Tasks_to_perform_on_dataset.pdf  
  inflating: ./data/Employe_Performance_dataset.csv  


### Pakete installieren

In [9]:
!pip install pandas


## Datensatz


In [10]:
import pandas as pd

# Pfad zur entpackten CSV-Datei
file_path = "./data/Employe_Performance_dataset.csv"

# CSV in DataFrame laden
df = pd.read_csv(file_path)

# Erste Zeilen ausgeben
print(df.head())


   ID                Name  Age  Gender Department  Salary Joining Date  \
0   1        Cory Escobar   48  Female         HR    5641   2015-05-03   
1   2     Timothy Sanchez   25   Other      Sales    4249   2020-11-09   
2   3        Chad Nichols   57   Other      Sales    3058   2019-02-12   
3   4  Christine Williams   58  Female         IT    5895   2017-09-08   
4   5        Amber Harris   35   Other         IT    4317   2020-02-15   

   Performance Score  Experience    Status     Location  Session  
0                2.0          16    Active     New York    Night  
1                2.0          11  Inactive  Los Angeles  Evening  
2                NaN           1  Inactive     New York  Morning  
3                2.0          13  Inactive  Los Angeles  Evening  
4                5.0          16  Inactive     New York  Evening  



## Hashing – Integritätsprüfung mit SHA2

1. Datensatz mit SHA-256 verschlüsseln.


In [11]:
import hashlib

# DataFrame als CSV-String umwandeln
csv_string = df.to_csv(index=False)

# SHA-256 (SHA-2 Familie)
sha256_full = hashlib.sha256(csv_string.encode()).hexdigest()
print("SHA-256-Hash des gesamten Dokuments:", sha256_full)

SHA-256-Hash des gesamten Dokuments: 6d0e81ad3755d4ede9af1bfa28df9f4114e69f6b98995e19b888afbb70ed54b7


2. Eine Änderung im Datensatz simulieren.

In [12]:
df_mod1 = df.copy()

# Salary in der ersten Zeile leicht ändern
df_mod1.loc[0, "Salary"] = 5642

# Ausgabe
df_mod1.head()

,ID,Name,Age,Gender,Department,Salary,Joining Date,Performance Score,Experience,Status,Location,Session
0,1,Cory Escobar,48,Female,HR,5642,2015-05-03,2.0,16,Active,New York,Night
1,2,Timothy Sanchez,25,Other,Sales,4249,2020-11-09,2.0,11,Inactive,Los Angeles,Evening
2,3,Chad Nichols,57,Other,Sales,3058,2019-02-12,NaN,1,Inactive,New York,Morning
3,4,Christine Williams,58,Female,IT,5895,2017-09-08,2.0,13,Inactive,Los Angeles,Evening
4,5,Amber Harris,35,Other,IT,4317,2020-02-15,5.0,16,Inactive,New York,Evening


3. Die Veränderung wird beim Vergleich der beiden Hashwerte sofort sichtbar. Ein Manipulations- oder Fälschungsversuch würde dadurch auffallen – etwa, wenn innerhalb eines Unternehmens eine Datei dupliziert und verändert weitergeleitet wird oder wenn man eine Datei aus dem Internet herunterlädt und den bekannten Original-Hashwert zum Abgleich heranzieht.

In [13]:
# SHA-256-Hash des geänderten DataFrames berechnen
sha256_full_modified = hashlib.sha256(df_mod1.to_csv(index=False).encode()).hexdigest()

print("SHA-256-Hash des geänderten Dokuments:", sha256_full_modified)

SHA-256-Hash des geänderten Dokuments: 13e645f0a622c58a97aa97d8620c262a2081aca0d5a4597f2aec91ef198dc0fa



## Hashing – Integritätsprüfung mit SHA3

1. Datensatz mit SHA3-256 verschlüsseln.


In [14]:
import hashlib

# DataFrame als CSV-String umwandeln
csv_string = df.to_csv(index=False)

# SHA3-256 (SHA-3 Familie, Keccak)
sha3_256_full = hashlib.sha3_256(csv_string.encode()).hexdigest()
print("SHA3-256-Hash des gesamten Dokuments:", sha3_256_full)


SHA3-256-Hash des gesamten Dokuments: 76e6e26835985a7b8b67357581e661baca2cbc010b9792062087c50e0f643497


2. Eine Änderung am Datensatz simulieren

In [15]:
df_mod2 = df.copy()

# Salary in der ersten Zeile leicht ändern
df_mod2.loc[0, "Salary"] = 5646

# Ausgabe
df_mod2.head()

,ID,Name,Age,Gender,Department,Salary,Joining Date,Performance Score,Experience,Status,Location,Session
0,1,Cory Escobar,48,Female,HR,5646,2015-05-03,2.0,16,Active,New York,Night
1,2,Timothy Sanchez,25,Other,Sales,4249,2020-11-09,2.0,11,Inactive,Los Angeles,Evening
2,3,Chad Nichols,57,Other,Sales,3058,2019-02-12,NaN,1,Inactive,New York,Morning
3,4,Christine Williams,58,Female,IT,5895,2017-09-08,2.0,13,Inactive,Los Angeles,Evening
4,5,Amber Harris,35,Other,IT,4317,2020-02-15,5.0,16,Inactive,New York,Evening


3. Die Veränderung wird beim Vergleich der beiden Hashwerte sofort sichtbar. Ein Manipulations- oder Fälschungsversuch würde dadurch auffallen – etwa, wenn innerhalb eines Unternehmens eine Datei dupliziert und verändert weitergeleitet wird oder wenn man eine Datei aus dem Internet herunterlädt und den bekannten Original-Hashwert zum Abgleich heranzieht.

In [16]:
# SHA3-256-Hash des geänderten DataFrames berechnen
sha3_256_full_modified = hashlib.sha3_256(df_mod2.to_csv(index=False).encode()).hexdigest()

print("SHA3-256-Hash des geänderten Dokuments:", sha3_256_full_modified)

SHA3-256-Hash des geänderten Dokuments: 0c6eba2716f8723f2137e550ed2818a6e745ebe057d33f4bb0539757a43447a9



## Symmetrische Verschlüsselung – AES

1. Es wird simuliert, wie ein Mitarbeiter eine Datei verschlüsselt und anschließend die verschlüsselte Version abspeichert.

In [17]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
import os

# Passwort (bekannt bei Absender und Empfänger)
password = b"MeinGeheimesPasswort"

# Salt und IV erzeugen
salt = os.urandom(16)
iv = os.urandom(16)

# Schlüssel aus Passwort ableiten
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend()
)
key = kdf.derive(password)

# Datei laden
with open(file_path, "rb") as f:
    plaintext = f.read()

# Padding
padded = plaintext + b" " * ((16 - len(plaintext) % 16) % 16)

# Verschlüsseln
cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
encryptor = cipher.encryptor()
ciphertext = encryptor.update(padded) + encryptor.finalize()

# Speicherpfad festlegen
# colab: save_dir = os.path.expanduser("~/content/Krypto")
save_dir = os.path.expanduser("~/Documents/Krypto")
os.makedirs(save_dir, exist_ok=True)

enc_file_path = os.path.join(save_dir, "employee_data.enc")

# Salt + IV + Ciphertext zusammen speichern
with open(enc_file_path, "wb") as f:
    f.write(salt + iv + ciphertext)

print("Datei erfolgreich mit AES verschlüsselt!")
print("Gespeichert unter:", enc_file_path)


Datei erfolgreich mit AES verschlüsselt!
Gespeichert unter: /root/Documents/Krypto/employee_data.enc


2. Der empfangende Mitarbeiter kann die Datei mit dem gemeinsam bekannten Passwort wieder entschlüsseln, sodass ausschließlich er Zugriff auf die Inhalte hat und die Daten sicher übertragen wurden.

In [18]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.kdf.pbkdf2 import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes
import os

# Passwort (muss dasselbe sein wie beim Verschlüsseln!)
password = b"MeinGeheimesPasswort"

# Speicherpfad für verschlüsselte Datei (muss existieren)
# colab: save_dir = os.path.expanduser("~/content/Krypto")
save_dir = os.path.expanduser("~/Documents/Krypto")
enc_file_path = os.path.join(save_dir, "employee_data.enc")

# Datei laden
with open(enc_file_path, "rb") as f:
    file_data = f.read()

# Salt, IV und Ciphertext extrahieren
salt = file_data[:16]
iv = file_data[16:32]
ciphertext = file_data[32:]

# Schlüssel aus Passwort und Salt ableiten
kdf = PBKDF2HMAC(
    algorithm=hashes.SHA256(),
    length=32,
    salt=salt,
    iterations=100000,
    backend=default_backend()
)
key = kdf.derive(password)

# Entschlüsseln
cipher = Cipher(algorithms.AES(key), modes.CBC(iv), backend=default_backend())
decryptor = cipher.decryptor()
decrypted = decryptor.update(ciphertext) + decryptor.finalize()
decrypted = decrypted.rstrip(b" ")  # Padding entfernen

# Entschlüsselte Datei speichern
dec_file_path = os.path.join(save_dir, "employee_data_decrypted.csv")
with open(dec_file_path, "wb") as f:
    f.write(decrypted)

print("Datei erfolgreich entschlüsselt!")
print("Gespeichert unter:", dec_file_path)


Datei erfolgreich entschlüsselt!
Gespeichert unter: /root/Documents/Krypto/employee_data_decrypted.csv


Ergebnis: Auf diese Weise können Mitarbeiter Dokumente verschlüsselt austauschen, die nur von Personen mit dem richtigen Passwort geöffnet werden können.

## Symmetrische Verschlüsselung – Blowfish

1. Es wird simuliert, wie ein Mitarbeiter eine Datei verschlüsselt und anschließend die verschlüsselte Version abspeichert.

In [19]:
!pip install pycryptodome

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 20.4 MB/s eta 0:00:00


In [20]:
from Crypto.Cipher import Blowfish
from Crypto.Protocol.KDF import PBKDF2
import os

# Passwort (bekannt bei Absender und Empfänger)
password = b"MeinGeheimesPasswort"

# Salt erzeugen
salt = os.urandom(16)

# Schlüssel aus Passwort ableiten (max. 56 Byte für Blowfish)
key = PBKDF2(password, salt, dkLen=32, count=100000)

# IV erzeugen (8 Byte, weil Blowfish Blockgröße = 64 Bit)
iv = os.urandom(8)

# Datei laden
with open(file_path, "rb") as f:
    plaintext = f.read()

# Padding (auf 8 Byte auffüllen)
padded = plaintext + b" " * ((8 - len(plaintext) % 8) % 8)

# Verschlüsseln
cipher = Blowfish.new(key, Blowfish.MODE_CBC, iv=iv)
ciphertext = cipher.encrypt(padded)

# Speicherpfad festlegen
# Für Colab: save_dir = "/content/Krypto"
save_dir = os.path.expanduser("~/Documents/Krypto")
os.makedirs(save_dir, exist_ok=True)

enc_file_path = os.path.join(save_dir, "employee_data_blowfish.enc")

# Salt + IV + Ciphertext zusammen speichern
with open(enc_file_path, "wb") as f:
    f.write(salt + iv + ciphertext)

print("Datei erfolgreich mit Blowfish verschlüsselt!")
print("Gespeichert unter:", enc_file_path)

Datei erfolgreich mit Blowfish verschlüsselt!
Gespeichert unter: /root/Documents/Krypto/employee_data_blowfish.enc


2. Der empfangende Mitarbeiter kann die Datei mit dem gemeinsam bekannten Passwort wieder entschlüsseln, sodass ausschließlich er Zugriff auf die Inhalte hat und die Daten sicher übertragen wurden.

In [21]:
from Crypto.Cipher import Blowfish
from Crypto.Protocol.KDF import PBKDF2
import os

# Passwort (muss dasselbe sein wie beim Verschlüsseln!)
password = b"MeinGeheimesPasswort"

# Speicherpfad für verschlüsselte Datei (muss existieren)
# Colab: save_dir = "/content/Krypto"
save_dir = os.path.expanduser("~/Documents/Krypto")
enc_file_path = os.path.join(save_dir, "employee_data_blowfish.enc")

# Datei laden
with open(enc_file_path, "rb") as f:
    file_data = f.read()

# Salt (16 Byte) + IV (8 Byte) + Ciphertext extrahieren
salt = file_data[:16]
iv = file_data[16:24]
ciphertext = file_data[24:]

# Schlüssel aus Passwort + Salt ableiten (max. 56 Byte für Blowfish)
key = PBKDF2(password, salt, dkLen=32, count=100000)

# Entschlüsseln
cipher = Blowfish.new(key, Blowfish.MODE_CBC, iv=iv)
decrypted = cipher.decrypt(ciphertext)
decrypted = decrypted.rstrip(b" ")  # Padding entfernen

# Entschlüsselte Datei speichern
dec_file_path = os.path.join(save_dir, "employee_data_blowfish_decrypted.csv")
with open(dec_file_path, "wb") as f:
    f.write(decrypted)

print("Datei erfolgreich entschlüsselt!")
print("Gespeichert unter:", dec_file_path)


Datei erfolgreich entschlüsselt!
Gespeichert unter: /root/Documents/Krypto/employee_data_blowfish_decrypted.csv


Ergebnis: Auf diese Weise können Mitarbeiter Dokumente verschlüsselt austauschen, die nur von Personen mit dem richtigen Passwort geöffnet werden können.